# AMI

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.pyplot import figure

import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
def calc_prevalence(y_actual):
    return (sum(y_actual)/len(y_actual))

def calc_specificity(y_actual, y_pred, thresh):
    # calculates specificity
    return sum((y_pred < thresh) & (y_actual == 0)) /sum(y_actual ==0)

def print_report(y_actual, y_pred, thresh):
    
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > thresh))
    recall = recall_score(y_actual, (y_pred > thresh))
    precision = precision_score(y_actual, (y_pred > thresh))
    specificity = calc_specificity(y_actual, y_pred, thresh)
    f1score = f1_score(y_actual, y_pred > thresh)
    
    print('AUC:%.3f'%auc)
    print('accuracy:%.3f'%accuracy)
    print('recall:%.3f'%recall)
    print('precision:%.3f'%precision)
    print('specificity:%.3f'%specificity)
    print('f1_score:%.3f'%f1score)

    print(' ')
    return auc, accuracy, recall, precision, specificity, f1score

## data

In [ ]:
df = pd.read_csv('./AMI.csv')

### exploratory data analysis

In [ ]:
sns.distplot(df['Age'])
plt.show()

In [ ]:
# 单列特征与标签的关系
# 不同年龄段，患心脏病和不患的分布图
pd.crosstab(df.Age,df.expire_flag).plot(kind='bar',figsize=(6,3))
plt.title("HeartDiseaseAndAge")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.savefig("HeartDiseaseAndAge.png")
plt.show()

In [ ]:
pd.crosstab(df.Sex,df.expire_flag).plot(kind="bar",figsize=(4,3),color=['#1CA53B',"#AA1111"])
plt.title("HeartDiseaseAndAge")
plt.xlabel("Sex")
plt.xticks(rotation=0)
plt.legend(["Haven not Disease","Have Disease"])
plt.ylabel("Frequency")
plt.savefig("HeartDiseaseAndAge.png")
plt.show()

In [ ]:
sns.violinplot(x="expire_flag",y="Age",hue="Sex",data=df,split=True)
plt.show()

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.3, random_state=1)

y_train = train_data.expire_flag
X_train = train_data.drop(['expire_flag'], axis=1)

y_test = test_data.expire_flag
X_test = test_data.drop(['expire_flag'], axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
ss = StandardScaler()
X_train_s = ss.fit_transform(X_train)
X_test_s = ss.transform(X_test)

mm = MinMaxScaler()
X_train_m = mm.fit_transform(X_train)
X_test_m = mm.transform(X_test)

## Table-One

In [ ]:
train_c = train_data.copy()
test_c = test_data.copy()

In [ ]:
train_c['group'] = 1
test_c['group'] = 2
merge_df = pd.concat([train_c, test_c])

In [ ]:
categorical = ['Sex', 'Race', 'Insurance', 'Marital_status', 'First_careunit', 'Hypertension', 
               'MI_history', 'PCI','Bypass','Drug', '90Status']

nonnormal =  list(set(columns).difference(set(categorical)))
print(nonnormal)

In [ ]:
import tableone

In [ ]:
groupby = '90Status'
mytable = tableone.TableOne(
    df, columns, categorical, groupby, 
    nonnormal, pval=True, label_suffix=True,htest_name=True)

In [ ]:
mytable.to_csv('./table_one.csv')

## Full variable model comparison

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN 
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import accuracy_score  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale  

# SVM
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm 

from sklearn.naive_bayes import MultinomialNB

import xgboost as xgb

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

### Logistic Regression

In [ ]:
lr = LogisticRegression(random_state = 42)
lr.fit(X_train, y_train)

In [ ]:
y_train_preds = lr.predict_proba(X_train)[:,1]
y_valid_preds = lr.predict_proba(X_test)[:,1]

print('Logistic Regression')
print('Training:')
thresh = 0.5
lr_train_auc, lr_train_accuracy, lr_train_recall, \
    lr_train_precision, lr_train_specificity, lr_train_f1 = print_report(y_train,y_train_preds, thresh)
print('Validation:')
lr_valid_auc, lr_valid_accuracy, lr_valid_recall, \
    lr_valid_precision, lr_valid_specificity, lr_test_f1 = print_report(y_test,y_valid_preds, thresh)

### Adaboost

In [ ]:
def ada_cv(n_estimators, learning_rate):
    val = cross_val_score(
        AdaBoostClassifier(n_estimators=int(n_estimators), 
                           learning_rate=learning_rate,
                           random_state=42), 
        X_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return val

ada_bo = BayesianOptimization(
    ada_cv,
    {'n_estimators': (1, 500),
     'learning_rate': (0.01, 0.999)}
)

ada_bo.maximize()

In [ ]:
adaboost = AdaBoostClassifier(learning_rate=min(ada_bo.max['params']['learning_rate'], 0.999), 
                              n_estimators=int(ada_bo.max['params']['n_estimators']), 
                              random_state=42)
adaboost.fit(X_train, y_train)

In [ ]:
y_train_preds = adaboost.predict_proba(X_train)[:,1]
y_valid_preds = adaboost.predict_proba(X_test)[:,1]
thresh = 0.5

print('Adaboost')
print('Training:')
ada_train_auc, ada_train_accuracy, ada_train_recall, \
    ada_train_precision, ada_train_specificity, ada_train_f1 = print_report(y_train,y_train_preds, thresh)
print('Validation:')
ada_valid_auc, ada_valid_accuracy, ada_valid_recall, \
    ada_valid_precision, ada_valid_specificity, ada_test_f1 = print_report(y_test,y_valid_preds, thresh)

In [ ]:
y_train_preds = adaboost.predict_proba(X_train)[:,1]
y_valid_preds = adaboost.predict_proba(X_test)[:,1]
thresh = 0.5

print('Adaboost')
print('Training:')
ada_train_auc, ada_train_accuracy, ada_train_recall, \
    ada_train_precision, ada_train_specificity, ada_train_f1 = print_report(y_train,y_train_preds, thresh)
print('Validation:')
ada_valid_auc, ada_valid_accuracy, ada_valid_recall, \
    ada_valid_precision, ada_valid_specificity, ada_test_f1 = print_report(y_test,y_valid_preds, thresh)

### KNN

In [ ]:
def knn_cv(n_neighbors):
    val = cross_val_score(KNN(n_neighbors = int(n_neighbors)),
        X_train_m, y_train, scoring='roc_auc', cv=5
    ).mean()
    return val

knn_bo = BayesianOptimization(
    knn_cv,
    {'n_neighbors': (1, 500)}
)

knn_bo.maximize()

In [ ]:
knn=KNN(n_neighbors = int(knn_bo.max['params']['n_neighbors']))
knn.fit(scale(X_train_m), y_train)

In [ ]:
y_train_preds = knn.predict_proba(scale(X_train_m))[:,1]
y_valid_preds = knn.predict_proba(scale(X_test_m))[:,1]
thresh = 0.5
print('KNN')
print('Training:')
knn_train_auc, knn_train_accuracy, knn_train_recall, \
    knn_train_precision, knn_train_specificity, knn_train_f1 = print_report(y_train,y_train_preds, thresh)
print('Validation:')
knn_valid_auc, knn_valid_accuracy, knn_valid_recall, \
    knn_valid_precision, knn_valid_specificity, knn_test_f1 = print_report(y_test,y_valid_preds, thresh)

### SVM

In [ ]:
svm = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
svm.fit(X_train_m, y_train)

In [ ]:
y_train_preds = svm.decision_function(X_train_m)
y_valid_preds = svm.decision_function(X_test_m)
thresh = 0.5

print('SVM')
print('Training:')
svm_train_auc, svm_train_accuracy, svm_train_recall, \
    svm_train_precision, svm_train_specificity, svm_train_f1 = print_report(y_train,y_train_preds, thresh)
print('Validation:')
svm_valid_auc, svm_valid_accuracy, svm_valid_recall, \
    svm_valid_precision, svm_valid_specificity, svm_test_f1 = print_report(y_test,y_valid_preds, thresh)

### MultinomialNB

In [ ]:
mnb = MultinomialNB(alpha=0.001)
mnb.fit(X_train_m, y_train)

In [ ]:
y_train_preds = mnb.predict_proba(X_train_m)[:, 1]
y_valid_preds = mnb.predict_proba(X_test_m)[:, 1]
thresh = 0.5
print('MNB')
print('Training:')
mnb_train_auc, mnb_train_accuracy, mnb_train_recall, \
    mnb_train_precision, mnb_train_specificity, mnb_train_f1 = print_report(y_train,y_train_preds, thresh)
print('Validation:')
mnb_valid_auc, mnb_valid_accuracy, mnb_valid_recall, \
    mnb_valid_precision, mnb_valid_specificity, mnb_test_f1 = print_report(y_test,y_valid_preds, thresh)

### XGBoost

In [ ]:
def xgb_cv(n_estimators, max_depth, learning_rate):
    val = cross_val_score(
        xgb.XGBRegressor(max_depth=int(max_depth), 
                         learning_rate=min(learning_rate, 0.999), 
                         n_estimators=int(n_estimators),
                         random_state=42), 
        X_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return val

xgb_bo = BayesianOptimization(
    xgb_cv,
    {'n_estimators': (10, 1000),
     'learning_rate': (0.001, 0.999),
     'max_depth': (1, 150)}
)

xgb_bo.maximize()

In [ ]:
learning_rate = xgb_bo.max['params']['learning_rate']
max_depth = xgb_bo.max['params']['max_depth']
n_estimators = xgb_bo.max['params']['n_estimators']

xgre = xgb.XGBRegressor(n_estimators=int(n_estimators),
                        learning_rate=min(learning_rate, 0.999),  # float
                        max_depth=int(max_depth),
                        random_state=2)
xgre.fit(X_train, y_train, verbose=True)

In [ ]:
y_train_preds = xgre.predict(X_train)
y_valid_preds = xgre.predict(X_test)
thresh=0.5
print('XGBoost')
print('Training:')
xgre_train_auc, xgre_train_accuracy, xgre_train_recall, \
    xgre_train_precision, xgre_train_specificity, xgre_train_f1 = print_report(y_train,y_train_preds, thresh)
print('Validation:')
xgre_valid_auc, xgre_valid_accuracy, xgre_valid_recall, \
    xgre_valid_precision, xgre_valid_specificity, xgre_test_f1 = print_report(y_test,y_valid_preds, thresh)

### Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train_m, y_train)

In [ ]:
y_train_preds = nb.predict_proba(X_train_m)[:,1]
y_valid_preds = nb.predict_proba(X_test_m)[:,1]

print('Naive Bayes')
print('Training:')
nb_train_auc, nb_train_accuracy, nb_train_recall, nb_train_precision, nb_train_specificity, nb_train_f1 =print_report(y_train,y_train_preds, thresh)
print('Validation:')
nb_valid_auc, nb_valid_accuracy, nb_valid_recall, nb_valid_precision, nb_valid_specificity, nb_test_f1 = print_report(y_test,y_valid_preds, thresh)

### Decision Tree

In [ ]:
from sklearn.model_selection import cross_val_score

score_list = []
for i in range(40):
    tree = DecisionTreeClassifier(max_depth =i, random_state = 42)
    score = cross_val_score(tree, X_train, y_train, cv=10, scoring='roc_auc').mean()
    score_list.append(score)

import matplotlib.pyplot as plt

plt.plot(range(0,len(score_list)), score_list)
plt.xlabel('max_depth')
plt.ylabel('score')   # 通过图像选择最好的参数
plt.show()

In [ ]:
tree = DecisionTreeClassifier(max_depth =12, random_state = 42)
tree.fit(X_train_m, y_train)

In [ ]:
y_train_preds = tree.predict_proba(X_train_m)[:,1]
y_valid_preds = tree.predict_proba(X_test_m)[:,1]

print('Decision Tree')
print('Training:')
tree_train_auc, tree_train_accuracy, tree_train_recall, tree_train_precision, tree_train_specificity, tree_train_f1 =print_report(y_train,y_train_preds, thresh)
print('Validation:')
tree_valid_auc, tree_valid_accuracy, tree_valid_recall, tree_valid_precision, tree_valid_specificity, tree_test_f1 = print_report(y_test,y_valid_preds, thresh)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RandomForestClassifier

In [ ]:
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        ), X_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return val

rf_bo = BayesianOptimization(
    rf_cv,
    {'n_estimators': (10, 200),
     'min_samples_split': (2, 25),
     'max_features': (0.1, 0.999),
     'max_depth': (5, 10)}
)

rf_bo.maximize()

In [ ]:
max_features = rf_bo.max['params']['max_features']
max_depth = rf_bo.max['params']['max_depth']
n_estimators = rf_bo.max['params']['n_estimators']
min_samples_split = rf_bo.max['params']['min_samples_split']

rf = RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        )

rf.fit(X_train, y_train)

In [ ]:
y_train_preds = rf.predict_proba(X_train)[:,1]
y_valid_preds = rf.predict_proba(X_test)[:,1]
thresh = 0.5
print('Random Forest')
print('Training:')
rf_train_auc, rf_train_accuracy, rf_train_recall, rf_train_precision, rf_train_specificity, rf_train_f1 =print_report(y_train,y_train_preds, thresh)
print('Validation:')
rf_valid_auc, rf_valid_accuracy, rf_valid_recall, rf_valid_precision, rf_valid_specificity, rf_test_f1 = print_report(y_test,y_valid_preds, thresh)

### Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from bayes_opt import BayesianOptimization
def gbc_cv(n_estimators, max_depth, learning_rate):
    val = cross_val_score(
        GradientBoostingClassifier(max_depth=int(max_depth), 
                         learning_rate=min(learning_rate, 0.999), 
                         n_estimators=int(n_estimators)), 
        X_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return val

gbc_bo = BayesianOptimization(
    gbc_cv,
    {'n_estimators': (1, 50),
     'learning_rate': (0.001, 0.999),
     'max_depth': (1, 5)}
)

gbc_bo.maximize()

In [ ]:
gbc = GradientBoostingClassifier(
    n_estimators=int(gbc_bo.max['params']['n_estimators']), 
    learning_rate=min(gbc_bo.max['params']['learning_rate'], 0.999), 
    max_depth=int(gbc_bo.max['params']['max_depth']), random_state=42)
gbc.fit(X_train, y_train)

In [ ]:
y_train_preds = gbc.predict_proba(X_train)[:,1]
y_valid_preds = gbc.predict_proba(X_test)[:,1]

print('Gradient Boosting Classifier')
print('Training:')
gbc_train_auc, gbc_train_accuracy, gbc_train_recall, gbc_train_precision, gbc_train_specificity, gbc_train_f1 = print_report(y_train,y_train_preds, thresh)
print('Validation:')
gbc_valid_auc, gbc_valid_accuracy, gbc_valid_recall, gbc_valid_precision, gbc_valid_specificity, gbc_test_f1 = print_report(y_test,y_valid_preds, thresh)

### Integrate model evaluation results and compare them

In [ ]:
df_results = pd.DataFrame(
    {'classifier':['Ada', 'Ada', 'KNN', 'KNN', 'SVM', 'SVM', 'MNB', 'MNB','XGB', 'XGB',
                   'LR', 'LR', 'NB','NB','DT','DT','RF','RF','GB','GB'],
     'data_set':['train','valid']*10,
     'auc':[ada_train_auc,  ada_valid_auc,
            knn_train_auc,  knn_valid_auc,
            svm_train_auc,  svm_valid_auc, 
            mnb_train_auc,  mnb_valid_auc, 
            xgre_train_auc, xgre_valid_auc, 
            lr_train_auc,   lr_valid_auc,
            nb_train_auc,   nb_valid_auc,
            tree_train_auc, tree_valid_auc,
            rf_train_auc,   rf_valid_auc,
            gbc_train_auc,  gbc_valid_auc],
     'accuracy':[ada_train_accuracy, ada_valid_accuracy,
                 knn_train_accuracy, knn_valid_accuracy,
                 svm_train_accuracy, svm_valid_accuracy, 
                 mnb_train_accuracy, mnb_valid_accuracy, 
                 xgre_train_accuracy, xgre_valid_accuracy, 
                 lr_train_accuracy,lr_valid_accuracy,
                 nb_train_accuracy,nb_valid_accuracy,
                 tree_train_accuracy,tree_valid_accuracy,
                 rf_train_accuracy,rf_valid_accuracy,
                 gbc_train_accuracy,gbc_valid_accuracy],
     'recall':[ada_train_recall, ada_valid_recall,
               knn_train_recall, knn_valid_recall,
               svm_train_recall, svm_valid_recall, 
               mnb_train_recall, mnb_valid_recall, 
               xgre_train_recall, xgre_valid_recall, 
               lr_train_recall,lr_valid_recall,
               nb_train_recall,nb_valid_recall,
               tree_train_recall,tree_valid_recall,
               rf_train_recall,rf_valid_recall,
               gbc_train_recall,gbc_valid_recall],  
     'precision':[ada_train_precision, ada_valid_precision, 
                  knn_train_precision, knn_valid_precision, 
                  svm_train_precision, svm_valid_precision, 
                  mnb_train_precision, mnb_valid_precision, 
                  xgre_train_precision, xgre_valid_precision, 
                  lr_train_precision,lr_valid_precision,
                  nb_train_precision,nb_valid_precision,
                  tree_train_precision,tree_valid_precision,
                  rf_train_precision,rf_valid_precision,
                  gbc_train_precision,gbc_valid_precision],   
     'specificity':[ada_train_specificity, ada_valid_specificity, 
                    knn_train_specificity, knn_valid_specificity,
                    svm_train_specificity, svm_valid_specificity, 
                    mnb_train_specificity, mnb_valid_specificity, 
                    xgre_train_specificity, xgre_valid_specificity, 
                    lr_train_specificity,lr_valid_specificity,
                    nb_train_specificity,nb_valid_specificity,
                    tree_train_specificity,tree_valid_specificity,
                    rf_train_specificity,rf_valid_specificity,
                    gbc_train_specificity,gbc_valid_specificity],
     'f1_score':[ada_train_f1, ada_test_f1, 
                 knn_train_f1, knn_test_f1,
                 svm_train_f1, svm_test_f1, 
                 mnb_train_f1, mnb_test_f1, 
                 xgre_train_f1, xgre_test_f1, 
                 lr_train_f1,lr_test_f1,
                 nb_train_f1,nb_test_f1,
                 tree_train_f1,tree_test_f1,
                 rf_train_f1,rf_test_f1,
                 gbc_train_f1,gbc_test_f1]})

In [ ]:
df_results.loc[df_results.data_set == 'train']  

In [ ]:
df_results.loc[df_results.data_set == 'train'].to_csv('./figure/10_model_train_evaluate.csv')

In [ ]:
df_results.loc[df_results.data_set == 'valid']

In [ ]:
df_results.loc[df_results.data_set == 'valid'].to_csv('./figure/10_model_valid_evaluate.csv')

In [ ]:
ax = sns.barplot(x="classifier", y="recall", hue="data_set", data=df_results)
ax.set_xlabel('Classifier',fontsize = 15)
ax.set_ylabel('AUC', fontsize = 15)
ax.tick_params(labelsize=15)

# Put the legend out of the figure
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize = 15)
plt.show()

## Choose a model to rebuild

Make K-fold calibration ROC chart, PR chart, variable importance chart

In [ ]:
from sklearn import metrics

In [ ]:
def rf_cv(n_estimators, min_samples_split, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
#             max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        ), X_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return val

rf_bo = BayesianOptimization(
    rf_cv,
    {'n_estimators': (10, 1000),
     'min_samples_split': (2, 10),
     'max_depth': (2, 150)}
)

rf_bo.maximize()

In [ ]:
from rfpimp import plot_corr_heatmap

In [ ]:
viz = plot_corr_heatmap(X_train, figsize=(15,10))
viz.view()

In [ ]:
# max_features = rf_bo.max['params']['max_features']
max_depth = rf_bo.max['params']['max_depth']
n_estimators = rf_bo.max['params']['n_estimators']
min_samples_split = rf_bo.max['params']['min_samples_split']

rf = RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
#             max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            n_jobs = -1,
            oob_score = True,
            bootstrap = True,
            random_state=2
        )

rf.fit(X_train, y_train)

y_valid_preds = rf.predict_proba(X_test)[:,1]
rf_valid_auc_base = roc_auc_score(y_test, y_valid_preds)

print('Validation AUC:%.3f'%(rf_valid_auc_base))

In [ ]:
from sklearn.metrics import r2_score
from rfpimp import permutation_importances

def r2(rf, X_train, y_train):
    return r2_score(y_train, rf.predict(X_train))

perm_imp_rfpimp = permutation_importances(rf, X_train, y_train, r2)

In [ ]:
perm_imp_rfpimp

In [ ]:
X_train.columns

In [ ]:
rf.fit(X_train.drop('Age', axis = 1), y_train)
drop_col_score = rf.score(X_train.drop('Age', axis = 1), y_train)

In [ ]:
drop_col_score

In [ ]:
from sklearn.base import clone 

def drop_col_feat_imp(model, X_train, y_train, random_state = 2):

    # clone the model to have the exact same specification as the one initially trained
    model_clone = clone(model)
    # set random_state for comparability
    model_clone.random_state = random_state
    # training and scoring the benchmark model
    model_clone.fit(X_train, y_train)
    benchmark_score = cross_val_score(model_clone, X_train, y_train, cv=5, scoring='recall').mean()
    
    # list for storing feature importances
    importances = []

    # iterating over all columns and storing feature importance (difference between benchmark and new model)
    for col in X_train.columns:
        model_clone = clone(model)
        model_clone.random_state = random_state
        model_clone.fit(X_train.drop(col, axis = 1), y_train)
        drop_col_score = cross_val_score(model_clone, X_train.drop(col, axis = 1), 
                                         y_train, cv=5, scoring='recall').mean()
        importances.append(benchmark_score - drop_col_score)

    importances_df = pd.DataFrame({"feature_importance": importances,
              "feature_name": X_train.columns.values})
    return importances_df

In [ ]:
fea_data = drop_col_feat_imp(rf, X_train, y_train)

In [ ]:
fea_data

In [ ]:
# feature_importance = rf.feature_importances_
# fea_data = pd.DataFrame({"feature_importance": feature_importance,
#               "feature_name": X_train.columns.values})

fea_data_sort = fea_data.sort_values(by='feature_importance',ascending=False)
pos = np.arange(fea_data_sort.shape[0])+.5

features_list = X_train.columns.values

figure(num=None, figsize=(4, 17), dpi=300)
plt.style.use('fivethirtyeight')


plt.barh(pos, fea_data_sort.feature_importance[::-1], align='center')
plt.yticks(pos, fea_data_sort.feature_name[::-1])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
# plt.savefig('./figure/all_feature_VariableImportance.tif', bbox_inches='tight', dpi=600, transparent=True)
plt.show()

### Evaluate

In [ ]:
y_train_preds = rf.predict_proba(X_train)[:,1]
y_valid_preds = rf.predict_proba(X_test)[:,1]
thresh = 0.5
print('Random Forest')
print('Training:')
rf_train_auc, rf_train_accuracy, rf_train_recall, rf_train_precision, rf_train_specificity, rf_train_f1 =print_report(y_train,y_train_preds, thresh)
print('Validation:')
rf_valid_auc, rf_valid_accuracy, rf_valid_recall, rf_valid_precision, rf_valid_specificity, rf_test_f1 = print_report(y_test,y_valid_preds, thresh)

In [ ]:
rf_results = pd.DataFrame(
    {'data_set':['Train','Test'] * 6,
     'evaluator': ['AUC', 'AUC', 'accuracy', 'accuracy', 'recall', 'recall', 
                   'precision', 'precision', 'specificty', 'specificty', 
                   'f1_socre', 'f1_socre'],
     'value': [rf_train_auc, rf_valid_auc, rf_train_accuracy, rf_valid_accuracy,
              rf_train_recall, rf_valid_recall, rf_train_precision, rf_valid_precision,
              rf_train_specificity, rf_valid_specificity, rf_train_f1, rf_test_f1]})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white")

from matplotlib import rcParams

%config InlineBackend.figure_formats = ['svg']
# 更改字体字号 10.5=五号字
rcParams['font.size']=20
rcParams['svg.fonttype']='none'
rcParams['font.sans-serif']=['Times New Roman']
rcParams['mathtext.fontset']='stix'
rcParams['axes.grid']=True
rcParams['axes.axisbelow']=True
rcParams['grid.linestyle']='--'
rcParams['xtick.direction']='in'
rcParams['ytick.direction']='in'

### Vari Import

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]
feat_labels = X_train.columns.values
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

In [ ]:
feature_importance = rf.feature_importances_
fea_data = pd.DataFrame({"feature_importance": feature_importance,
              "feature_name": X_train.columns.values})

fea_data_sort = fea_data.sort_values(by='feature_importance',ascending=False)
pos = np.arange(fea_data_sort.shape[0])+.5

features_list = X_train.columns.values

figure(num=None, figsize=(4, 17), dpi=300)
plt.style.use('fivethirtyeight')

plt.barh(pos, fea_data_sort.feature_importance[::-1], align='center')
plt.yticks(pos, fea_data_sort.feature_name[::-1])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.savefig('./figure/all_feature_VariableImportance.tif', bbox_inches='tight', dpi=600, transparent=True)
plt.show()

- 随机森林是无法很好的处理分类变量的，会倾向于类别多的变量

In [ ]:
feature_importance = rf.feature_importances_
fea_data = pd.DataFrame({"feature_importance": feature_importance,
              "feature_name": X_train.columns.values})

fea_data_sort = fea_data.sort_values(by='feature_importance',ascending=False)[:20]
pos = np.arange(fea_data_sort.shape[0])+.5

# features_list = X_train.columns.values

figure(num=None, figsize=(4, 7), dpi=300)
plt.style.use('fivethirtyeight')

plt.barh(pos, fea_data_sort.feature_importance[::-1], align='center')
plt.yticks(pos, fea_data_sort.feature_name[::-1])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.savefig('./figure/all_feature_VariableImportance_20.tif', bbox_inches='tight', dpi=600, transparent=True)
plt.show()

### ROC

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import auc

In [ ]:
# Run classifier with cross-validation and plot ROC curves
kf = KFold(n_splits=10, shuffle=True, random_state=42)    # 定义分成几个组

classifier = RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
#             max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        )

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

target = df.expire_flag.values
data = df.drop(['expire_flag'], axis=1).values
figure(num=None, figsize=(7, 6), dpi=300)
# plt.style.use("bmh")


i = 0
for train_index, test_index in kf.split(data):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    probas_ = classifier.fit(X_train, y_train).predict_proba(X_test)[:,1]
    
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y_test, probas_)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.3f)' % (i, roc_auc))

    i += 1

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

## 确定最佳阈值
right_index = (mean_tpr + (1 - mean_fpr) - 1).tolist()
index = right_index.index(max(right_index))

tpr_val = mean_tpr[index]
fpr_val = mean_fpr[index]

# 画ROC
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.3f $\pm$ %0.3f)' % (mean_auc, std_auc),
         lw=2)

# 添加最佳截断值点
plt.plot([fpr_val], [tpr_val], 'ro',)
plt.annotate('('+'%0.3f'%fpr_val+', '+'%0.3f'%tpr_val+')',
             xy=(fpr_val,tpr_val),xytext=(fpr_val+0.01,tpr_val+0.01))

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='darkgray', alpha=.3,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right", fontsize=12)
plt.savefig('./figure/All_feature_10_fold_ROC.tif', dpi=600, bbox_inches='tight', transparent=True)
plt.show()

### PR

In [ ]:
from sklearn.metrics import precision_recall_curve, auc
from scipy import interpolate

In [ ]:
# Run classifier with cross-validation and plot ROC curves
kf = KFold(n_splits=10, shuffle=True, random_state=42)    # 定义分成几个组

classifier = RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
#             max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        )

precisions = []
prs = []
mean_recall = np.linspace(0, 1, 100)

target = df.expire_flag.values
data = df.drop(['expire_flag'], axis=1).values
figure(num=None, figsize=(7, 6), dpi=300)
# plt.style.use("bmh")

i = 0
for train_index, test_index in kf.split(data):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    probas_ = classifier.fit(X_train, y_train).predict_proba(X_test)[:,1]
    
    # Compute ROC curve and area the curve
    precision, recall, _ = precision_recall_curve(y_test, probas_)
    auprc = auc(recall, precision)
    plt.plot(recall, precision, lw=1, alpha=0.3, label='PR fold %d (AUPRC = %0.3f)' % (i, auprc))
    
#     print("recall: ", recall.shape)
#     print("precision: ", precision.shape)
#     print(np.interp(mean_recall, recall, precision))
    precisions.append(interpolate.interp1d(recall, precision)(mean_recall))
    precisions[-1][0] = 1.0
    prs.append(auprc)
   
    i += 1

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_precision = np.mean(precisions, axis=0)
mean_precision[-1] = 0.0
mean_pr = auc(mean_recall, mean_precision)
std_pr = np.std(prs)

# 画ROC
plt.plot(mean_recall, mean_precision, color='b',
         label=r'Mean PR (AUPRC = %0.3f $\pm$ %0.3f)' % (mean_pr, std_pr),
         lw=2)

# 添加平衡点
ba_point = 0.750
plt.plot([ba_point], [ba_point], 'ro',)
plt.annotate('(balance: '+'%0.3f'%ba_point+')',
             xy=(ba_point,ba_point),xytext=(ba_point+0.01,ba_point+0.01))

std_precision = np.std(precisions, axis=0)
precisions_upper = np.minimum(mean_precision + std_precision, 1)
precisions_lower = np.maximum(mean_precision - std_precision, 0)

plt.fill_between(mean_recall, precisions_lower, precisions_upper, color='darkgray', alpha=.3,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR')
plt.legend(loc="lower left", fontsize=12)
plt.savefig('./figure/All_feature_10_fold_PR.tif', dpi=600, bbox_inches='tight', transparent=True)
plt.show()

## Lasso

### feature_sel

In [ ]:
df = pd.read_csv('./use_df.csv')

train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)

y_train = train_data.expire_flag
X_train = train_data.drop(['expire_flag'], axis=1)

y_test = test_data.expire_flag
X_test = test_data.drop(['expire_flag'], axis=1)

In [ ]:
from sklearn.linear_model import Lasso, LassoCV

alphas = 10**np.linspace(-4, -2, 100)
lasso_cofficients = []

In [ ]:
for alpha in alphas: 
    lasso = Lasso(alpha = alpha, normalize= True, max_iter= 10000) 
    lasso.fit(X_train, y_train) 
    lasso_cofficients.append(lasso.coef_)

In [ ]:
from matplotlib.pylab import mpl
mpl.rcdefaults()
plt.rcParams[ 'font.sans-serif'] = [ 'Microsoft YaHei']
plt.rcParams[ 'axes.unicode_minus'] = False

figure(num=None, figsize=(8, 5), dpi=300)

plt.plot(alphas, lasso_cofficients, lw=2)
plt.xscale( 'log')
plt.axis( 'tight')
plt.title( 'Relationship between alpha and LASSO regression coefficient')
plt.xlabel( 'Log Alpha')
plt.ylabel( 'Cofficients')
plt.grid(axis="both")
plt.savefig('./figure/lasso.tif', dpi=600, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
lasso_cv = LassoCV(alphas = alphas, normalize= True, cv = 10)
lasso_cv.fit(X_train, y_train)

lasso_best_alpha = lasso_cv.alpha_
lasso_best_alpha

In [ ]:
lasso = Lasso(alpha = lasso_best_alpha, normalize= True, max_iter= 10000)
lasso.fit(X_train, y_train)

print(lasso.coef_)

lasso_predict = lasso.predict(X_train)
roc = roc_auc_score(y_train, lasso_predict)
roc

In [ ]:
from sklearn.model_selection import cross_val_score
import matplotlib
def rmse_cv(model):
    rmse= cross_val_score(model, X_train, y_train, scoring="roc_auc", cv = 5)
    return(rmse)

In [ ]:
X_train

In [ ]:
model_lasso = lasso
print(model_lasso.alpha)
print(model_lasso.coef_)

coef = pd.Series(model_lasso.coef_, index = X_train.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

print(rmse_cv(model_lasso).mean())
num = int(sum(coef != 0) / 2)
imp_coef = pd.concat([coef.sort_values().head(num),
                     coef.sort_values().tail(num)])

figure(num=None, figsize=(4, 8), dpi=300)
imp_coef.plot(kind = "barh")
plt.grid(axis='both')
plt.title("Coefficients in the Lasso Model")
plt.savefig('./figure/lasso_coefficients.tif', dpi=600, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
lasso_cols = coef[coef.values != 0].index.to_list()
lasso_cols.append('expire_flag')

In [ ]:
print(lasso_cols)

In [ ]:
new_cols = ['Age', 'Sex'
            'BMI', 'HR', 'Glucose','SysBP',
            'CK', 
            'APSIII', 
            'Creatinine', 'eGFR', 'UrineOutput', 
            'RDW', 'Albumin', 'Lymphocytes', 
            'AG', 'Potassium', 'Sodium', 'Chloride', 'Platlet', 
            'Hemoglobin',
            'expire_flag']

In [ ]:
lasso_df

In [ ]:
lasso_df = pd.DataFrame(df, columns=new_cols)
train_data, test_data = train_test_split(lasso_df, test_size=0.3, random_state=42)

y_train = train_data.expire_flag
X_train = train_data.drop(['expire_flag'], axis=1)

y_test = test_data.expire_flag
X_test = test_data.drop(['expire_flag'], axis=1)

In [ ]:
def rf_cv(n_estimators, min_samples_split, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
#             max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        ), X_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return val

rf_bo = BayesianOptimization(
    rf_cv,
    {'n_estimators': (10, 1000),
     'min_samples_split': (2, 10),
#      'max_features': (0.001, 0.999),
     'max_depth': (2, 150)}
)

rf_bo.maximize()

In [ ]:
# max_features = rf_bo.max['params']['max_features']
max_depth = rf_bo.max['params']['max_depth']
n_estimators = rf_bo.max['params']['n_estimators']
min_samples_split = rf_bo.max['params']['min_samples_split']

rf = RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
#             max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        )

rf.fit(X_train, y_train)

In [ ]:
X_train.shape

### evaluate

In [ ]:
y_train_preds = rf.predict_proba(X_train)[:,1]
y_valid_preds = rf.predict_proba(X_test)[:,1]
thresh = 0.5
print('Random Forest')
print('Training:')
rf_train_auc, rf_train_accuracy, rf_train_recall, rf_train_precision, rf_train_specificity, rf_train_f1 =print_report(y_train,y_train_preds, thresh)
print('Validation:')
lasso_rf_valid_auc, lasso_rf_valid_accuracy, lasso_rf_valid_recall, lasso_rf_valid_precision, lasso_rf_valid_specificity, lasso_rf_test_f1 = print_report(y_test,y_valid_preds, thresh)

In [ ]:
# 比较全变量和lasso选择变量的结果
labels = ['AUC', 'Accuracy', 'Recall', 'Precision', 'Specificity', 'F1_Score'] 
lasso_means = [lasso_rf_valid_auc.round(2), lasso_rf_valid_accuracy.round(2), lasso_rf_valid_recall.round(2), lasso_rf_valid_precision.round(2), np.float64(lasso_rf_valid_specificity).round(2), lasso_rf_test_f1.round(2)] 
all_means = [rf_valid_auc.round(2), rf_valid_accuracy.round(2), rf_valid_recall.round(2), rf_valid_precision.round(2), np.float64(rf_valid_specificity).round(2), rf_test_f1.round(2)]  
x = np.arange(len(labels))  

# the label locations 
width = 0.35  
plt.rcParams['figure.figsize'] = (10.0, 4.0) # 设置figure_size尺寸
plt.rcParams['font.sans-serif']=['Times New Roman']
plt.rcParams['font.size']=14
plt.rcParams['svg.fonttype']='none'


rects1 = plt.bar(x - width/2, all_means, width, label='All') 
rects2 = plt.bar(x + width/2, lasso_means, width, label='Lasso')  
# Add some text for labels, title and custom x-axis tick labels, etc. 


def autolabel(rects):     
    """Attach a text label above each bar in *rects*, displaying its height."""     
    for rect in rects:         
        height = rect.get_height()         
        plt.annotate('{}'.format(height),                     
                    xy=(rect.get_x() + rect.get_width() / 2, height),                     
                    xytext=(0, 3),  # 3 points vertical offset                     
                    textcoords="offset points",                     
                    ha='center', 
                    va='bottom')  
autolabel(rects1) 
autolabel(rects2) 
ax1=plt.gca()
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)

ax1.set_xlabel('Score', fontsize=16)
ax1.set_ylabel('Evaluation Index', fontsize=16)
# plt.title("Random Forest ", fontsize=16)
plt.xticks(ticks=x, labels=labels, rotation=45, fontsize=16) 
plt.yticks(fontsize=16) 

plt.legend(loc='lower right', fontsize =16)   


plt.savefig('./figure/RF_ALL_Lasso_Evaluate.tif', bbox_inches='tight', dpi=600, transparent=True)

plt.show()

### Vari Impor

In [ ]:
feature_importance = rf.feature_importances_
fea_data = pd.DataFrame({"feature_importance": feature_importance,
              "feature_name": X_train.columns.values})

fea_data_sort = fea_data.sort_values(by='feature_importance',ascending=False)
pos = np.arange(fea_data_sort.shape[0])+.5

features_list = X_train.columns.values

figure(num=None, figsize=(4, 7), dpi=300)
plt.style.use('fivethirtyeight')

plt.barh(pos, fea_data_sort.feature_importance[::-1], align='center')
plt.yticks(pos, fea_data_sort.feature_name[::-1])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.savefig('./figure/lasso_feature_VariableImportance.tif', bbox_inches='tight', dpi=600, transparent=True)
plt.show()

In [ ]:
feature_importance = rf.feature_importances_
fea_data = pd.DataFrame({"feature_importance": feature_importance,
              "feature_name": X_train.columns.values})

fea_data_sort = fea_data.sort_values(by='feature_importance',ascending=False)[:20]
pos = np.arange(fea_data_sort.shape[0])+.5

# features_list = X_train.columns.values

figure(num=None, figsize=(4, 7), dpi=300)
plt.style.use('fivethirtyeight')

plt.barh(pos, fea_data_sort.feature_importance[::-1], align='center')
plt.yticks(pos, fea_data_sort.feature_name[::-1])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.savefig('./figure/Lasso_feature_VariableImportance_20.pdf', bbox_inches='tight', dpi=600, transparent=True)
# plt.show()

### ROC

In [ ]:
# Run classifier with cross-validation and plot ROC curves
kf = KFold(n_splits=10, shuffle=True, random_state=42)    # 定义分成几个组

classifier = RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
#             max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=2
        )

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

target = df.expire_flag.values
data = df.drop(['expire_flag'], axis=1).values
figure(num=None, figsize=(7, 6), dpi=300)
# plt.style.use("bmh")


i = 0
for train_index, test_index in kf.split(data):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    probas_ = classifier.fit(X_train, y_train).predict_proba(X_test)[:,1]
    
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y_test, probas_)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.3f)' % (i, roc_auc))

    i += 1

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

## 确定最佳阈值
right_index = (mean_tpr + (1 - mean_fpr) - 1).tolist()
index = right_index.index(max(right_index))

tpr_val = mean_tpr[index]
fpr_val = mean_fpr[index]

# 画ROC
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.3f $\pm$ %0.3f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

# 添加最佳截断值点
plt.plot([fpr_val], [tpr_val], 'ro',)
plt.annotate('('+'%0.3f'%fpr_val+', '+'%0.3f'%tpr_val+')',
             xy=(fpr_val,tpr_val),xytext=(fpr_val+0.01,tpr_val+0.01))

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)

plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='darkgray', alpha=.3,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right", fontsize=12)
plt.savefig('./figure/Lasso_10_fold_ROC.tif', dpi=600, bbox_inches='tight', transparent=True)
plt.show()

### PR

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

classifier = RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_depth=int(max_depth),
            random_state=2
        )

precisions = []
prs = []
mean_recall = np.linspace(0, 1, 100)

target = df.expire_flag.values
data = df.drop(['expire_flag'], axis=1).values
figure(num=None, figsize=(7, 6), dpi=300)

i = 0
for train_index, test_index in kf.split(data):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = target[train_index], target[test_index]
    probas_ = classifier.fit(X_train, y_train).predict_proba(X_test)[:,1]
    
    precision, recall, _ = precision_recall_curve(y_test, probas_)
    auprc = auc(recall, precision)
    plt.plot(recall, precision, lw=1, alpha=0.3, label='PR fold %d (AUPRC = %0.3f)' % (i, auprc))

    precisions.append(interpolate.interp1d(recall, precision)(mean_recall))
    precisions[-1][0] = 1.0
    prs.append(auprc)
   
    i += 1

plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_precision = np.mean(precisions, axis=0)
mean_precision[-1] = 0.0
mean_pr = auc(mean_recall, mean_precision)
std_pr = np.std(prs)

plt.plot(mean_recall, mean_precision, color='b',
         label=r'Mean PR (AUPRC = %0.3f $\pm$ %0.3f)' % (mean_pr, std_pr),
         lw=2, alpha=.8)

ba_point = 0.751
plt.plot([ba_point], [ba_point], 'ro',)
plt.annotate('(balance: '+'%0.3f'%ba_point+')',
             xy=(ba_point,ba_point),xytext=(ba_point+0.01,ba_point+0.01))

std_precision = np.std(precisions, axis=0)
precisions_upper = np.minimum(mean_precision + std_precision, 1)
precisions_lower = np.maximum(mean_precision - std_precision, 0)

plt.fill_between(mean_recall, precisions_lower, precisions_upper, color='darkgray', alpha=.3,
                 label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR')
plt.legend(loc="lower left", fontsize=12)
plt.savefig('./figure/Lasso_10_fold_PR.tif', dpi=600, bbox_inches='tight', transparent=True)
plt.show()

## Lasso_SHAP

In [ ]:
df = pd.read_csv('./use_df.csv')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
con_cols = ['Age', 'Sex',
            'BMI', 'HR', 'Glucose','SysBP',
            'CK', 
            'APSIII', 
            'Creatinine', 'eGFR', 'UrineOutput', 
            'RDW', 'Albumin', 'Lymphocytes', 
            'AG', 'Potassium', 'Sodium', 'Chloride', 'Platlet', 
            'Hemoglobin',
            'expire_flag']
len(con_cols)

In [ ]:
lasso_df = pd.DataFrame(df, columns=con_cols)
train_data, test_data = train_test_split(lasso_df, test_size=0.3, random_state=42)

y_train = train_data.expire_flag
X_train = train_data.drop(['expire_flag'], axis=1)

y_test = test_data.expire_flag
X_test = test_data.drop(['expire_flag'], axis=1)

In [ ]:
def rf_cv(n_estimators, min_samples_split, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_depth=int(max_depth),
            random_state=2
        ), X_train, y_train, scoring='roc_auc', cv=5
    ).mean()
    return val

rf_bo = BayesianOptimization(
    rf_cv,
    {'n_estimators': (10, 1000),
     'min_samples_split': (2, 10),
     'max_depth': (2, 150)}
)

rf_bo.maximize()

In [ ]:
max_depth = 40
n_estimators = 944
min_samples_split = 2

rf = RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_depth=int(max_depth),
            random_state=2
        )

rf.fit(X_train, y_train)

y_valid_preds = rf.predict_proba(X_test)[:,1]
rf_valid_auc_base = roc_auc_score(y_test, y_valid_preds)

print('Validation AUC:%.3f'%(rf_valid_auc_base))

In [ ]:
y_train_preds = rf.predict_proba(X_train)[:,1]
y_valid_preds = rf.predict_proba(X_test)[:,1]
thresh = 0.5
print('Random Forest')
print('Training:')
_, _, _, _, _, _ =print_report(y_train,y_train_preds, thresh)
print('Validation:')
_, _, _, _, _, _ = print_report(y_test,y_valid_preds, thresh)

In [ ]:
import shap

In [ ]:
explainer = shap.Explainer(rf)
shap_values = explainer.shap_values(X_test)[1]

In [ ]:
X_test.columns

In [ ]:
waterfall_shap = explainer(test_data.loc[test_data['expire_flag'] == 1][:20].drop(['expire_flag'], axis=1))[:,:,1]

In [ ]:
for i in range(20):
    fig = plt.figure(figsize=(6,4))

    shap.plots.waterfall(waterfall_shap[i], max_display=10, show=False)
    
    fig.set_facecolor('white')
    fig.savefig('./figure/waterfall_label_1_%s.tif'%str(i), bbox_inches="tight", dpi=600, transparent=True)

In [ ]:
con_cols = ['Age', 
            'BMI', 'HR', 'Glucose','SysBP',
            'CK', 
            'APSIII', 
            'Creatinine', 'eGFR', 'UrineOutput', 
            'RDW', 'Albumin', 'Lymphocytes', 
            'AG', 'Potassium', 'Sodium', 'Chloride', 'Platlet', 
            'Hemoglobin',
            'expire_flag']

In [ ]:
interaction_list = [['APSIII', 'Age'], 
                    ['APSIII', 'Sex'],
                    ['SysBP','APSIII'],
                    ['APSIII', 'eGFR'], 
                    ['RDW', 'APSIII'],
                    ['Albumin', 'APSIII'],
                    ['Lymphocytes', 'APSIII'],
                    ['Sodium', 'APSIII'],
                    ['AG', 'APSIII'], 
['Potassium', 'APSIII'], ['Chloride', 'APSIII'], ['Hemoglobin', 'APSIII']
                   ]

In [ ]:
for inter in interaction_list:
    print(inter)
    fig = plt.figure(figsize=(6,4)) 
    ax = fig.subplots(1,1)

    ax.axhline(y=0, ls="-",c="red", linewidth=2)
    ax.grid(linewidth=1,alpha=0.3)  
    shap.dependence_plot(inter[0], shap_values, X_train, interaction_index=inter[1], ax=ax, show=False)

    fig.set_facecolor('white')
    name = "_".join(inter)
    fig.savefig('./figure/interction_%s.tif'%name, bbox_inches="tight", dpi=600, transparent=True)


    fig = plt.figure(figsize=(6,4)) 
    ax = fig.subplots(1,1)

    ax.axhline(y=0, ls="-",c="red", linewidth=2)
    ax.grid(linewidth=1,alpha=0.3)  
    shap.dependence_plot(inter[1], shap_values, X_train, interaction_index=inter[0], ax=ax, show=False)

    fig.set_facecolor('white')
    inter.reverse()
    name = "_".join(inter)
    fig.savefig('./figure/interction_%s.tif'%name, bbox_inches="tight", dpi=600, transparent=True)

In [ ]:
shap.dependence_plot('Creatinine', shap_values, X_test, 
                     interaction_index='AG', 
                     show=False)

In [ ]:
X_test

In [ ]:
n = 2
print('n: %d, y: %d'%(n, y_test.iloc[n]))
# shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[n], X_test.iloc[n], matplotlib=True, show=False)
plt.savefig('./figure/force_plot_label1.tif', bbox_inches="tight", dpi=600, transparent=True)

In [ ]:
n = 0
print('n: %d, y: %d'%(n, y_test.iloc[n]))
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[n], X_test.iloc[n], matplotlib=True, show=False)
plt.savefig('./figure/force_plot_label0.tif', bbox_inches="tight", dpi=600, transparent=True)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
plt.title('Feature importance of SHAP')
plt.savefig('./figure/summary_plot_bar.tif', bbox_inches="tight", dpi=600, transparent=True)

In [ ]:
shap.summary_plot(shap_values, X_test, show=False)
plt.title('SHAP beeswarm plot')
plt.savefig('./figure/summary_plot.tif', bbox_inches="tight", dpi=600, transparent=True)

In [ ]:
import statsmodels.api as sm

In [ ]:
con_cols.remove('expire_flag')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white")

from matplotlib import rcParams

%config InlineBackend.figure_formats = ['svg']
rcParams['font.size']=20
rcParams['svg.fonttype']='none'
rcParams['font.sans-serif']=['Times New Roman']
rcParams['mathtext.fontset']='stix'
rcParams['axes.grid']=True
rcParams['axes.axisbelow']=True
rcParams['grid.linestyle']='--'
rcParams['xtick.direction']='in'
rcParams['ytick.direction']='in'

In [ ]:
for feature_ in con_cols:

    fig = plt.figure(figsize=(6,4)) 

    idx = np.where(X_test.columns==feature_)[0][0]
    x = X_test.iloc[:,idx]
    y_sv = shap_values[:,idx]
    lowess = sm.nonparametric.lowess(y_sv, x, frac=.4)  

    ax = fig.subplots(1,1)

    ax.plot(*list(zip(*lowess)), color="red", ls=':', linewidth=4)
    ax.axhline(y=0, ls="-",c="red", linewidth=2)
#     ax.set_title('%s SHAP dependence plot'%feature_)
    ax.grid(linewidth=1,alpha=0.3)  
    # ax.set_xticklabels(ax.get_xticklabels(),fontsize=3)

    shap.dependence_plot(feature_, shap_values, X_test, ax=ax, interaction_index=None, show=False)
    
    fig.set_facecolor('white')
    fig.savefig('./figure/dependence_plot_%s.tif'%feature_, bbox_inches="tight", dpi=600, transparent=True)